In [1]:
from sentiment_analyzer import get_sentiment_dictionary_from_csv_path

In [ ]:
data_path = "C:/Users/Asus/Desktop/ttds-proj/backend/data/"
# outlet_folders = ["bbc", "gbn", "ind", "tele"]
outlet_folders = ["tele"]
output_file_path = "sentiment_dictionary/sentiment_dictionary.json"
sentiment_dictionary = {}

# Iterate over each outlet folder
for outlet_folder in outlet_folders:
    # Construct the path to the current outlet folder
    folder_path = os.path.join(data_path, outlet_folder)
    # List all files in the current outlet folder
    all_file_paths = os.listdir(folder_path)

    # Iterate over each file in the current outlet folder
    for file_name in tqdm(all_file_paths, desc=outlet_folder):
        # Construct the full path to the current file
        file_path = os.path.join(folder_path, file_name)
        # Ensure the file is a CSV before attempting to read it
        if file_path.endswith(".csv"):
            try:
                # Read the current CSV file into a pandas DataFrame
                sentiment_dictionary = get_sentiment_dictionary_from_csv_path(
                    file_path, csv_sentiment_dictionary=sentiment_dictionary
                )
            except Exception as e:
                print(f"Error with {file_path}: {e}")

with open(
    output_file_path, "wb"
) as file:  # 'wb' mode because orjson.dumps() returns bytes
    file.write(orjson.dumps(sentiment_dictionary))